In [1]:
import sys
sys.path.append("../src")
from config import *
from plugin_write_and_run import *

In [2]:
%%write_and_run ../src/replay_buffer.py
import numpy as np
import json
import sys
sys.path.append("../src")
from config import *

In [3]:
%%write_and_run -a ../src/replay_buffer.py

class ReplayBuffer():
    def __init__(self, env, buffer_capacity=BUFFER_CAPACITY, batch_size=BATCH_SIZE, min_size_buffer=MIN_SIZE_BUFFER):
        self.env = env
        self.buffer_capacity = buffer_capacity
        self.batch_size = batch_size
        self.min_size_buffer = min_size_buffer
        self.buffer_counter = 0
        self.n_games = 0
        
        self.states = np.zeros((self.buffer_capacity, env.observation_space.shape[0]))
        self.actions = np.zeros((self.buffer_capacity, env.action_space.shape[0]))
        self.rewards = np.zeros((self.buffer_capacity))
        self.next_states = np.zeros((self.buffer_capacity, env.observation_space.shape[0]))
        self.dones = np.zeros((self.buffer_capacity), dtype=bool)
        

        
    def __len__(self):
        return self.buffer_counter

    def add_record(self, state, action, reward, next_state, done):
        # Set index to zero if counter = buffer_capacity and start again (1 % 100 = 1 and 101 % 100 = 1) so we substitute the older entries
        index = self.buffer_counter % self.buffer_capacity

        self.states[index] = state
        self.actions[index] = action
        self.rewards[index] = reward
        self.next_states[index] = next_state
        self.dones[index] = done
        
        # Update the counter when record something
        self.buffer_counter += 1
    
    def check_buffer_size(self):
        return self.buffer_counter >= self.batch_size and self.buffer_counter >= self.min_size_buffer
    
    def update_n_games(self):
        self.n_games += 1
    
    def get_minibatch(self):
        # If the counter is less than the capacity we don't want to take zeros records, 
        # if the cunter is higher we don't access the record using the counter 
        # because older records are deleted to make space for new one
        buffer_range = min(self.buffer_counter, self.buffer_capacity)
        
        batch_index = np.random.choice(buffer_range, self.batch_size, replace=False)

        # Convert to tensors
        state = self.states[batch_index]
        action = self.actions[batch_index]
        reward = self.rewards[batch_index]
        next_state = self.next_states[batch_index]
        done = self.dones[batch_index]
        
        return state, action, reward, next_state, done
    
    def save(self, folder_name):
        """
        Save the replay buffer
        """
        if not os.path.isdir(folder_name):
            os.mkdir(folder_name)

        np.save(folder_name + '/states.npy', self.states)
        np.save(folder_name + '/actions.npy', self.actions)
        np.save(folder_name + '/rewards.npy', self.rewards)
        np.save(folder_name + '/dones.npy', self.dones)
        
        dict_info = {"buffer_counter": self.buffer_counter, "n_games": self.n_games}
        
        with open(folder_name + '/dict_info.json', 'w') as f:
            json.dump(dict_info, f)

    def load(self, folder_name):
        """
        Load the replay buffer
        """
        self.actions = np.load(folder_name + '/states.npy')
        self.actions = np.load(folder_name + '/actions.npy')
        self.rewards = np.load(folder_name + '/rewards.npy')
        self.dones = np.load(folder_name + '/dones.npy')
        
        with open(folder_name + '/dict_info.json', 'r') as f:
            dict_info = json.load(f)
        self.buffer_counter = dict_info["buffer_counter"]
        self.n_games = dict_info["n_games"]

In [4]:
import gym

In [5]:
env = gym.make(ENV_NAME)

In [6]:
rb = ReplayBuffer(env)

In [7]:
rb.buffer_counter

0

In [8]:
env.reset()

array([ 0.00292511,  1.4184309 ,  0.29626375,  0.33381495, -0.00338266,
       -0.06710813,  0.        ,  0.        ], dtype=float32)

In [9]:
env.action_space.shape

(2,)

In [10]:
action = np.array([-0.5, 0.1, 0.2, 0.3])

In [11]:
state, reward, done, _ = env.step(action)

In [12]:
for i in range(1000):
    rb.add_record(state, action, reward, state, done)

ValueError: could not broadcast input array from shape (4,) into shape (2,)

In [ ]:
rb.buffer_counter

In [ ]:
rb.dones

In [ ]:
state, action, reward, next_state, done = rb.get_minibatch()

In [ ]:
state.shape

In [ ]:
action.shape

In [ ]:
rb.buffer_counter

In [ ]:
reward.shape